In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style
from webdriver_manager.chrome import ChromeDriverManager
import re

# 二手房

In [2]:
driver = webdriver.Chrome()
url='https://esf.fang.com/house-a015277-b02655/'
driver.get(url)

In [3]:
results_1 = []
i = 0
Num_Pages = 20

while i < Num_Pages:
    house_list = driver.find_elements(By.CSS_SELECTOR, "dl.clearfix")
    for item in house_list:
        # 提取面积
        area_text = item.find_element(By.CSS_SELECTOR, "p.tel_shop").text
        # 使用正则表达式匹配面积
        area_match = re.search(r'(\d+(\.\d+)?)㎡', area_text)
        area = float(area_match.group(1)) if area_match else None
        
        # 提取总价
        total_price_text = item.find_element(By.CSS_SELECTOR, "dd.price_right span.red").text
        # 使用正则表达式匹配总价
        total_price_match = re.search(r'(\d+(\.\d+)?)万', total_price_text)
        total_price = float(total_price_match.group(1)) if total_price_match else None
        
        # 提取单价
        unit_price_text = item.find_element(By.CSS_SELECTOR, "dd.price_right span:nth-child(2)").text
        # 使用正则表达式匹配单价
        unit_price_match = re.search(r'(\d+(\.\d+)?)元/㎡', unit_price_text)
        unit_price = float(unit_price_match.group(1)) if unit_price_match else None
        
        # 将提取的结果添加到列表
        results_1.append({
            '面积(㎡)': area,
            '总价(元)': total_price,
            '单价(元/㎡)': unit_price
        })

    # 翻页
    try:
        next_page = driver.find_element('link text', '下一页')
        next_page.click()
        i += 1
    except NoSuchElementException:
        print(i)
        break

5


In [4]:
# 转化为Dataframe
df_1 = pd.DataFrame(results_1)
df_1['总价(元)'] = df_1['总价(元)']*10000
print(df_1)

      面积(㎡)       总价(元)  单价(元/㎡)
0     91.28   7200000.0  78878.0
1     57.00   3550000.0  62280.0
2     58.70   4800000.0  81771.0
3     56.70   3980000.0  70194.0
4    254.00  12550000.0  49409.0
..      ...         ...      ...
306   66.85   5200000.0  77786.0
307   67.90   5260000.0  77467.0
308   53.76   4400000.0  81846.0
309   56.70   3900000.0  68784.0
310   44.80   2400000.0  53570.0

[311 rows x 3 columns]


In [5]:
# 输出文件
df_1.to_csv('Beijing_Suzhouqiao_Sales.csv', index=False, encoding='utf-8')

# 租房

In [6]:
driver = webdriver.Chrome()
url='https://zu.fang.com/house-a015277-b02655/'
driver.get(url)

In [7]:
results_2 = []
i = 0
Num_Pages = 20

while i < Num_Pages:
    house_list = driver.find_elements(By.CSS_SELECTOR, "dl.list.hiddenMap.rel")
    for item in house_list:
        # 提取面积
        area_text = item.find_element(By.CSS_SELECTOR, "p.font15.mt12.bold").text
        # 使用正则表达式匹配面积
        area_match = re.search(r'(\d+(\.\d+)?)㎡', area_text)
        area = float(area_match.group(1)) if area_match else None
        
        # 提取总价
        price_text = item.find_element(By.CSS_SELECTOR, "span.price").text
        # 提取数字
        price = float(price_text)
        
        # 将提取的结果添加到列表
        results_2.append({
            '面积(㎡)': area,
            '总价(元/月)': price,
        })

    # 翻页
    try:
        next_page = driver.find_element('link text', '下一页')
        next_page.click()
        i += 1
    except NoSuchElementException:
        print(i)
        break

5


In [8]:
# 转化为Dataframe
df_2 = pd.DataFrame(results_2)
print(df_2)

     面积(㎡)  总价(元/月)
0     66.0  10200.0
1     74.0  15000.0
2     85.0  12500.0
3     63.0   7300.0
4     18.0   4260.0
..     ...      ...
300   66.0   7800.0
301   61.0   7803.0
302   56.0  11500.0
303   56.0   2700.0
304   60.0   7800.0

[305 rows x 2 columns]


In [9]:
# 输出文件
df_2.to_csv('Beijing_Suzhouqiao_Rent.csv', index=False, encoding='utf-8')